In [2]:
!pip install tensorflow  # This will install TensorFlow along with a compatible numpy version
!pip install transformers==4.33.0 tokenizers datasets accelerate
# Install tts with a version compatible with the numpy version required by TensorFlow, if available
# !pip install tts==compatible_version_here



In [1]:
import tensorflow as tf
import glob
import os
import shutil
import tqdm
import random
import matplotlib.pyplot as plt
import torch
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer

tf.config.list_physical_devices("GPU")

2024-02-18 15:19:53.222595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 15:19:53.222643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 15:19:53.267436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 15:19:53.365949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-18 15:19:54.470251: W tensorflow/compiler/tf2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

import os
import re
import glob

# Define the source and target directories
source_dir = '../data/input/clean_data'
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "filtered_dataset.txt")

# Ensure the target directory exists
os.makedirs(target_dir, exist_ok=True)

# Regex pattern to keep English letters, Pali diacritics, and standard punctuation
pattern = re.compile(r"[^a-zA-ZāīūṅñṭḍṇḷĀĪŪṄÑṬḌṆḶ\s.,!?()-]")

# Function to clean text by removing unwanted characters
def clean_text(text):
    return pattern.sub('', text)

# Find all .txt files in the source directory
paths = glob.glob(os.path.join(source_dir, "*.txt"))[:1000]  # Limit to first 1000 files

# Process and merge content of each file
with open(dataset_file, "w", encoding='utf-8') as output_file:
    for path in paths:
        with open(path, "r", encoding='utf-8') as file:
            for line in file:
                cleaned_line = clean_text(line.strip())
                if cleaned_line:
                    print(cleaned_line, file=output_file)

print("Corpus loaded and filtered.")


import os
import re
import glob
import shutil

# Define the source directories for copying .txt files
source_dirs = [
    "../data/input/text/books_converted",
    "../data/input/text/web_txt",
    "../data/input/audio_transcribed/transcribed_youtube",
    "../data/input/audio_transcribed/transscribed_audio_dharma"
]

# Define the target directory for copied .txt files
copy_target_dir = '../data/input/clean_data'

# Ensure the target directory exists
os.makedirs(copy_target_dir, exist_ok=True)

# Copy all .txt files from the source directories to the target directory
for src_dir in source_dirs:
    for txt_file in glob.glob(os.path.join(src_dir, "*.txt")):
        shutil.copy(txt_file, copy_target_dir)

# Now, set the source_dir to where we've copied the .txt files
source_dir = copy_target_dir

# Define the target directory for preprocessing output
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "filtered_dataset.txt")

# Ensure the preprocessing target directory exists
os.makedirs(target_dir, exist_ok=True)

# Regex pattern to keep English letters, Pali diacritics, and standard punctuation
pattern = re.compile(r"[^a-zA-ZāīūṅñṭḍṇḷĀĪŪṄÑṬḌṆḶ\s.,!?()-]")

# Function to clean text by removing unwanted characters
def clean_text(text):
    return pattern.sub('', text)

# Find all .txt files in the source directory for preprocessing
paths = glob.glob(os.path.join(source_dir, "*.txt"))[:1000]  # Limit to first 1000 files

# Process and merge content of each file
with open(dataset_file, "w", encoding='utf-8') as output_file:
    for path in paths:
        with open(path, "r", encoding='utf-8') as file:
            for line in file:
                cleaned_line = clean_text(line.strip())
                if cleaned_line:
                    print(cleaned_line, file=output_file)

print("Corpus loaded and filtered.")


# Clean Dataset: Convert to English Sentences

import spacy
import re

# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def is_likely_english(sentence):
    # Check if the sentence has at least one verb and one subject, common in English sentences
    has_verb = any(token.pos_ == 'VERB' for token in sentence)
    has_subject = any(token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass' for token in sentence)
    return has_verb and has_subject

def is_not_reference(line):
    # Filter out lines that resemble references or citations, e.g., "SN V," "T II b"
    return not re.search(r'\b(SN|T|MN|AN|D|SĀ|Ps|Vin|Vism)\s+[IVXLC]+', line)

def filter_sentences(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        for line in f_in:
            if is_not_reference(line):
                doc = nlp(line.strip())
                for sent in doc.sents:
                    if is_likely_english(sent) and len(sent.text.split()) > 5:  # Minimum word count check
                        f_out.write(sent.text + '\n')

# Specify the path to your input file and the desired output file
input_file = '../data/input/filtered_data/filtered_dataset.txt'
output_file = '../data/input/filtered_data/cleaned_dataset.txt'

# Filter the dataset
filter_sentences(input_file, output_file)

print("Dataset filtering complete. Check the cleaned dataset at:", output_file)



# Combined Cleaning and Filtering of Dataset

In [ ]:
import os
import re
import glob
import shutil
import spacy

# Define the source directories for copying .txt files
source_dirs = [
    "../data/input/text/books_converted",
    "../data/input/text/web_txt",
    "../data/input/audio_transcribed/transcribed_youtube",
    "../data/input/audio_transcribed/transscribed_audio_dharma"
]

# Define the first target directory for copied .txt files
copy_target_dir = '../data/input/clean_data'

# Ensure the first target directory exists
os.makedirs(copy_target_dir, exist_ok=True)

# Copy all .txt files from the source directories to the first target directory
for src_dir in source_dirs:
    for txt_file in glob.glob(os.path.join(src_dir, "*.txt")):
        shutil.copy(txt_file, copy_target_dir)

# Define the target directory for preprocessing output and the output file
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "filtered_dataset.txt")

# Ensure the preprocessing target directory exists
os.makedirs(target_dir, exist_ok=True)

# Regex pattern to keep English letters, Pali diacritics, and standard punctuation
pattern = re.compile(r"[^a-zA-ZāīūṅñṭḍṇḷĀĪŪṄÑṬḌṆḶ\s.,!?()-]")

# Function to clean text by removing unwanted characters
def clean_text(text):
    return pattern.sub('', text)

# Find and preprocess .txt files in the first target directory
paths = glob.glob(os.path.join(copy_target_dir, "*.txt"))[:1000]  # Limit to first 1000 files

# Process and merge content of each file
with open(dataset_file, "w", encoding='utf-8') as output_file:
    for path in paths:
        with open(path, "r", encoding='utf-8') as file:
            for line in file:
                cleaned_line = clean_text(line.strip())
                if cleaned_line:
                    print(cleaned_line, file=output_file)

# Load the English tokenizer, tagger, parser, NER, and word vectors from spaCy
nlp = spacy.load("en_core_web_sm")

def is_likely_english(sentence):
    # Check if the sentence has at least one verb and one subject
    has_verb = any(token.pos_ == 'VERB' for token in sentence)
    has_subject = any(token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass' for token in sentence)
    return has_verb and has_subject

def is_not_reference(line):
    # Filter out lines that resemble references or citations
    return not re.search(r'\b(SN|T|MN|AN|D|SĀ|Ps|Vin|Vism)\s+[IVXLC]+', line)

def filter_sentences(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        for line in f_in:
            if is_not_reference(line):
                doc = nlp(line.strip())
                for sent in doc.sents:
                    if is_likely_english(sent) and len(sent.text.split()) > 5:  # Minimum word count check
                        f_out.write(sent.text + '\n')

# Specify the path for the final output file
final_output_file = '../data/input/filtered_data/cleaned_dataset.txt'

# Filter the preprocessed dataset
filter_sentences(dataset_file, final_output_file)

print("Corpus loaded, filtered, and cleaned. Check the cleaned dataset at:", final_output_file)


# Produce Descriptive Statistics

In [2]:
import re

def format_number(number):
    """Format the number with underscores between hundreds."""
    return f"{number:,}".replace(",", "_")

def count_words(file_path):
    """Count words in a file and provide descriptive statistics."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            # Tokenize the text by non-alphanumeric characters
            words = re.findall(r'\b\w+\b', text.lower())
            word_count = len(words)
            unique_words = len(set(words))
            average_word_length = sum(len(word) for word in words) / word_count

            print(f"Total Words: {format_number(word_count)}")
            print(f"Unique Words: {format_number(unique_words)}")
            print(f"Average Word Length: {average_word_length:.2f} characters")
    except FileNotFoundError:
        print(f"The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    file_path = "../data/input/filtered_data/cleaned_dataset.txt"
    count_words(file_path)



Total Words: 9_171_086
Unique Words: 85_247
Average Word Length: 5.37 characters


In [3]:
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "cleaned_dataset.txt")

raw_datasets = load_dataset("text", data_files=dataset_file)
raw_datasets
# 333_056 lines
# previously around 43_000 lines

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 346626
    })
})

In [4]:
raw_datasets["train"][666]
# with HuggingFace datasets, every sample is a dictionary
# a sample from the dataset here is a paragraph
# the key is always "text"

{'text': 'Im assuming everybody is familiar with them.'}

In [5]:
# Create empty tokenizer and its trainer
tokenizer = Tokenizer(BPE(unk_token="[UNK]")) # subword tokenization and ways to merge them
trainer = BpeTrainer(vocab_size=5_000, special_tokens=["[UNK]", "[PAD]"])
# separates the tokens with a space
tokenizer.pre_tokenizer = Whitespace()

# Batch samples to speed up process
def batch_iterator(batch_size=1000):
    # the batch size is the number of samples that will be processed at once
    # the iterator will yield a batch of samples
    # yield is a keyword in Python that is used like return, except the function will return a generator
    # a generator is an iterator that generates one item at a time
    for i in range(0, len(raw_datasets["train"]), batch_size):
        yield raw_datasets["train"][i : i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(raw_datasets["train"]))

# Saves the tokenizer
# when downloading model, we download model, and the tokenizer
tokenizer.save("tokenizer.json")

# Load it fast
# speeds up the process
tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json") 
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

0

# Tokenize the Dataset

In [6]:
# start with sequence length of 256
# pads the sequences to the same length
sequence_length = 256

# takes a dictionary as input
def tokenize_function(example):
    # tokenize the text
    tokenized_example = tokenizer(
        example["text"],
        truncation=True,
        padding=True,
        max_length=sequence_length,
    )
    return {"input_ids": tokenized_example["input_ids"]}

# tokenize entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=raw_datasets["train"].column_names)

In [7]:
# data collator is used to batch the samples together
# data pump for training
# collate means to collect and combine
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Make the Model

In [8]:
model_config = GPT2Config(
    vocab_size=tokenizer.vocab_size, # the size of the vocabulary
    pad_token_id=tokenizer.pad_token_id, # the token id for padding
    n_ctx=sequence_length, # context length
    n_positions=sequence_length, # positions in context, the order of the tokens in sequence
    n_embd=512, # embedding dimension
    n_head=8, # number of heads in the multi-head attention models
    n_layer=6, # number of layers
)

model = GPT2LMHeadModel(model_config)
model

# wte: word token embeddings; which means the embeddings of the tokens
# wpe: word position embeddings; which means the embeddings of the positions of the tokens
# drop: dropout
# sequence length: 256
# dropout layer: 0.1
# dropout is a regularization technique; it prevents overfitting
# normalisation is done first, which differs to transformers
# normalisation means to scale the input to have a mean of 0 and a standard deviation of 1
# Conv1D means 1D convolution; convolutions are used to extract features from the input
# LayerNorm means layer normalisation; normalisation is used to improve the training of the model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(5000, 512)
    (wpe): Embedding(256, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=5000, bias=False)
)

In [9]:
# clear VRAM GPU cache
torch.cuda.empty_cache()

output_path = "output"

# Create the Trainer

training_args = TrainingArguments(
    output_dir=output_path, # output directory
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    #per_device_train_batch_size=16, # batch size for training per device (e.g. multiple GPUs), which took 8 minutes
    #per_device_train_batch_size=32 # double, because i was only using around 3GB of VRAM, which took 7.5 minutes
    per_device_train_batch_size=52 # increase, because i was only using around 6GB of VRAM with 32, which also took 7.5 minutes
    # now uses 7704MiB of VRAM
)

trainer = Trainer(
    model=model, # the model
    args=training_args, # training arguments
    data_collator=data_collator, # data collator
    train_dataset=tokenized_datasets["train"] # training dataset
)

# Train
trainer.train()

# Save
tokenizer.save_pretrained(output_path)
model.save_pretrained(output_path)

# TODO:
# You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/66660 [00:00<?, ?it/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 6.6431, 'learning_rate': 4.962496249624963e-05, 'epoch': 0.08}
{'loss': 6.0825, 'learning_rate': 4.924992499249925e-05, 'epoch': 0.15}
{'loss': 5.8659, 'learning_rate': 4.887488748874888e-05, 'epoch': 0.23}
{'loss': 5.6698, 'learning_rate': 4.84998499849985e-05, 'epoch': 0.3}
{'loss': 5.5196, 'learning_rate': 4.812481248124813e-05, 'epoch': 0.38}
{'loss': 5.3838, 'learning_rate': 4.774977497749775e-05, 'epoch': 0.45}
{'loss': 5.2824, 'learning_rate': 4.737473747374738e-05, 'epoch': 0.53}
{'loss': 5.1815, 'learning_rate': 4.6999699969997e-05, 'epoch': 0.6}
{'loss': 5.079, 'learning_rate': 4.6624662466246627e-05, 'epoch': 0.68}
{'loss': 5.019, 'learning_rate': 4.6249624962496254e-05, 'epoch': 0.75}
{'loss': 4.9651, 'learning_rate': 4.5874587458745876e-05, 'epoch': 0.83}
{'loss': 4.9066, 'learning_rate': 4.5499549954995503e-05, 'epoch': 0.9}
{'loss': 4.861, 'learning_rate': 4.5124512451245125e-05, 'epoch': 0.98}
{'loss': 4.7736, 'learning_rate': 4.474947494749475e-05, 'epoch': 1.

In [10]:
torch.cuda.empty_cache()

In [11]:
# Encode the conditioning tokens.
input_ids = tokenizer.encode("Jhana meditation is ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens.
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)

tensor([[1896,  363,   91]], device='cuda:0')
Jhana meditation is not a great deal of thinking and thinking, but it is very difficult to understand.................................................................................


In [12]:
# Assuming 'tokenizer' and 'model' are already defined

# Move model to GPU
model = model.cuda()

# Encode the conditioning tokens and move them to GPU
input_ids = tokenizer.encode("Jhana meditation is ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)

# Decode generated ids
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


tensor([[1896,  363,   91]], device='cuda:0')
Jhana meditation is a very important thing to do...........................................................................................


In [13]:
# Encode the conditioning tokens.
input_ids = tokenizer.encode("To practice jhana meditation, we sit ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens.
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)

tensor([[2490,  292,  456,  363,    5,  152,  481]], device='cuda:0')
To practice jhana meditation, we sit down to meditate and we can use the meditation as a meditation.................................................................................


In [14]:
# Assuming 'tokenizer' and 'model' are already defined

# Move model to GPU
model = model.cuda()

# Encode the conditioning tokens and move them to GPU
input_text = "Jhana meditation is "
input_ids = tokenizer.encode(input_text, return_tensors="pt").cuda()
print(f"Input text: {input_text}")
print(f"Encoded input_ids: {input_ids} on device {input_ids.device}")

# Generate more tokens
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
print(f"Raw generated ids: {generated_ids}")

# Decode generated ids
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(f"Decoded generated sequence: {generated_sequence}")


Input text: Jhana meditation is 
Encoded input_ids: tensor([[1896,  363,   91]], device='cuda:0') on device cuda:0
Raw generated ids: tensor([[1896,  363,   91,  530,   90,  148,    6, 1286,  771,    7,    7,    7,
            7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,
            7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,
            7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,
            7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,
            7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,
            7,    7,    7,    7,    7,  679,    7,    7,    7,    7,    7,    7,
            7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,    7,
          679,    7,    7,  679]], device='cuda:0')
Decoded generated sequence: Jhana meditation is called the mind - door process.................................................................... ().................

In [15]:
input_text = "Jhana meditation is "
print(f"Input text: '{input_text}'")


Input text: 'Jhana meditation is '


In [16]:
tokens = tokenizer.tokenize(input_text)
print(f"Tokens: {tokens}")


Tokens: ['Jhana', 'meditation', 'is']


In [17]:
input_ids = tokenizer.encode(input_text, return_tensors="pt")
print(f"Encoded input_ids: {input_ids}")
decoded_tokens = [tokenizer.decode([id]) for id in input_ids[0]]
print(f"Decoded tokens from input_ids: {decoded_tokens}")


Encoded input_ids: tensor([[1896,  363,   91]])
Decoded tokens from input_ids: ['Jhana', 'meditation', 'is']


In [18]:
print(f"Number of tokens: {len(tokens)}")
print(f"Number of input_ids: {input_ids.shape[1]}")


Number of tokens: 3
Number of input_ids: 3


In [19]:
print(f"Special tokens added: {tokenizer.build_inputs_with_special_tokens(tokens)}")


Special tokens added: ['Jhana', 'meditation', 'is']


In [20]:
attention_mask = tokenizer(input_text, return_tensors="pt")["attention_mask"]
print(f"Attention mask: {attention_mask}")


Attention mask: tensor([[1, 1, 1]])


In [21]:
print(f"Model device: {next(model.parameters()).device}")
print(f"Input ids device: {input_ids.device}")


Model device: cuda:0
Input ids device: cpu


In [22]:
# Ensure input_ids are on the same device as the model
input_ids = input_ids.to(next(model.parameters()).device)

# Now you can proceed with generating more tokens
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)

# Decode generated ids
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


Jhana meditation is a very interesting thing, very interesting thing.................................................................. ()..................... ()


In [23]:
print(f"Model device: {next(model.parameters()).device}")
print(f"Input ids device: {input_ids.device}")


Model device: cuda:0
Input ids device: cuda:0


In [15]:
# Load model locally

#model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=path)
#tokenizer = PreTrainedTokenizerFast(pretrained_model_name_or_path=path)